In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsOneClassifier
from sklearn.metrics import accuracy_score

In [3]:
file_path = r"C:\Users\dipak\OneDrive\desktop\github_ai\ml_with_python\three_module\Multi-class-Classification\Obesity_level_prediction_dataset.csv"

In [4]:
df = pd.read_csv(file_path)

In [5]:
df.sample(5)

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
1040,Male,33.081600,1.705617,83.016968,yes,yes,2.000000,2.797600,Sometimes,no,2.991671,no,2.148738,0.000000,no,Automobile,Overweight_Level_II
948,Female,31.626380,1.666023,72.906186,yes,yes,2.000000,3.000000,Sometimes,no,1.000000,no,0.000000,1.301385,Sometimes,Automobile,Overweight_Level_I
60,Female,21.000000,1.550000,49.000000,yes,yes,2.000000,3.000000,Sometimes,no,3.000000,no,3.000000,1.000000,Sometimes,Public_Transportation,Normal_Weight
599,Male,18.000000,1.716691,51.149283,yes,yes,1.813234,3.000000,Sometimes,no,1.274815,no,0.520407,1.673584,Sometimes,Public_Transportation,Insufficient_Weight
1108,Female,19.027417,1.588147,73.939268,yes,no,3.000000,3.362758,Sometimes,no,2.000000,no,2.892922,1.000000,Sometimes,Public_Transportation,Overweight_Level_II


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2111 entries, 0 to 2110
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Gender                          2111 non-null   object 
 1   Age                             2111 non-null   float64
 2   Height                          2111 non-null   float64
 3   Weight                          2111 non-null   float64
 4   family_history_with_overweight  2111 non-null   object 
 5   FAVC                            2111 non-null   object 
 6   FCVC                            2111 non-null   float64
 7   NCP                             2111 non-null   float64
 8   CAEC                            2111 non-null   object 
 9   SMOKE                           2111 non-null   object 
 10  CH2O                            2111 non-null   float64
 11  SCC                             2111 non-null   object 
 12  FAF                             21

In [7]:
df.describe()

,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE
count,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000
mean,24.312600,1.701677,86.586058,2.419043,2.685628,2.008011,1.010298,0.657866
std,6.345968,0.093305,26.191172,0.533927,0.778039,0.612953,0.850592,0.608927
min,14.000000,1.450000,39.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,19.947192,1.630000,65.473343,2.000000,2.658738,1.584812,0.124505,0.000000
50%,22.777890,1.700499,83.000000,2.385502,3.000000,2.000000,1.000000,0.625350
75%,26.000000,1.768464,107.430682,3.000000,3.000000,2.477420,1.666678,1.000000
max,61.000000,1.980000,173.000000,3.000000,4.000000,3.000000,3.000000,2.000000


Feature Scaling:

In [8]:
# Standardizing the continous numerical features:
continous_columns = df.select_dtypes(include=["float64"]).columns.tolist()

scaler = StandardScaler()
scaled_features = scaler.fit_transform(df[continous_columns])

In [ ]:
scaled_features_df = pd.DataFrame(
    scaled_features, columns=scaler.get_feature_names_out(continous_columns)
)
scaled_features_df.head()

,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE
0,-0.522124,-0.875589,-0.862558,-0.785019,0.404153,-0.013073,-1.188039,0.561997
1,-0.522124,-1.947599,-1.168077,1.088342,0.404153,1.618759,2.339750,-1.080625
2,-0.206889,1.054029,-0.366090,-0.785019,0.404153,-0.013073,1.163820,0.561997
3,0.423582,1.054029,0.015808,1.088342,0.404153,-0.013073,1.163820,-1.080625
4,-0.364507,0.839627,0.122740,-0.785019,-2.167023,-0.013073,-1.188039,-1.080625


In [10]:
# Combining with the original dataset
scaled_data = pd.concat(
    [df.drop(columns=continous_columns), scaled_features_df], axis=1
)

In [11]:
scaled_data.head()

,Gender,family_history_with_overweight,FAVC,CAEC,SMOKE,SCC,CALC,MTRANS,NObeyesdad,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE
0,Female,yes,no,Sometimes,no,no,no,Public_Transportation,Normal_Weight,-0.522124,-0.875589,-0.862558,-0.785019,0.404153,-0.013073,-1.188039,0.561997
1,Female,yes,no,Sometimes,yes,yes,Sometimes,Public_Transportation,Normal_Weight,-0.522124,-1.947599,-1.168077,1.088342,0.404153,1.618759,2.339750,-1.080625
2,Male,yes,no,Sometimes,no,no,Frequently,Public_Transportation,Normal_Weight,-0.206889,1.054029,-0.366090,-0.785019,0.404153,-0.013073,1.163820,0.561997
3,Male,no,no,Sometimes,no,no,Frequently,Walking,Overweight_Level_I,0.423582,1.054029,0.015808,1.088342,0.404153,-0.013073,1.163820,-1.080625
4,Male,no,no,Sometimes,no,no,Sometimes,Public_Transportation,Overweight_Level_II,-0.364507,0.839627,0.122740,-0.785019,-2.167023,-0.013073,-1.188039,-1.080625


In [12]:
# Identifying categorical columns

In [13]:
categorical_columns = scaled_data.select_dtypes(include=["object"]).columns.to_list()
categorical_columns.remove("NObeyesdad")

In [14]:
categorical_columns

['Gender',
 'family_history_with_overweight',
 'FAVC',
 'CAEC',
 'SMOKE',
 'SCC',
 'CALC',
 'MTRANS']

In [15]:
# Applying one hot encoding
encoder = OneHotEncoder(sparse_output=False, drop="first")

In [16]:
encoded_features = encoder.fit_transform(scaled_data[categorical_columns])

In [17]:
encoded_df = pd.DataFrame(
    encoded_features, columns=encoder.get_feature_names_out(categorical_columns)
)

In [18]:
prepped_data = pd.concat(
    [scaled_data.drop(columns=categorical_columns), encoded_df], axis=1
)

In [19]:
prepped_data["NObeyesdad"] = prepped_data["NObeyesdad"].astype("category").cat.codes
prepped_data.head()

,NObeyesdad,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE,Gender_Male,...,CAEC_no,SMOKE_yes,SCC_yes,CALC_Frequently,CALC_Sometimes,CALC_no,MTRANS_Bike,MTRANS_Motorbike,MTRANS_Public_Transportation,MTRANS_Walking
0,1,-0.522124,-0.875589,-0.862558,-0.785019,0.404153,-0.013073,-1.188039,0.561997,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,1,-0.522124,-1.947599,-1.168077,1.088342,0.404153,1.618759,2.339750,-1.080625,0.0,...,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,1,-0.206889,1.054029,-0.366090,-0.785019,0.404153,-0.013073,1.163820,0.561997,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,5,0.423582,1.054029,0.015808,1.088342,0.404153,-0.013073,1.163820,-1.080625,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,6,-0.364507,0.839627,0.122740,-0.785019,-2.167023,-0.013073,-1.188039,-1.080625,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [20]:
X = prepped_data.drop("NObeyesdad", axis=1)
y = prepped_data["NObeyesdad"]

In [21]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [22]:
model_ova = LogisticRegression(multi_class="ovr", max_iter=1000)
model_ova.fit(X_train, y_train)

c:\Users\dipak\.conda\envs\env2\Lib\site-packages\sklearn\linear_model\_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(max_iter=1000, multi_class='ovr')

In [23]:
y_pred_ova = model_ova.predict(X_test)

In [24]:
print("One-vs-all (OvA) Strategy")

One-vs-all (OvA) Strategy


In [25]:
print(f"Accuracy: {np.round(100*accuracy_score(y_test,y_pred_ova),2)}%")

Accuracy: 76.12%


In [26]:
model_ovo = OneVsOneClassifier(LogisticRegression(max_iter=1000))

In [27]:
model_ovo.fit(X_train, y_train)

OneVsOneClassifier(estimator=LogisticRegression(max_iter=1000))

In [28]:
y_pred_ovo = model_ovo.predict(X_test)

In [29]:
print("One-vs-One  (OvO) strategy")

One-vs-One  (OvO) strategy


In [30]:
print(f"Accuracy: {np.round(100*accuracy_score(y_test, y_pred_ovo),2)}%")

Accuracy: 92.2%


In [38]:
for i in [0.1, 0.3]:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=i, stratify=y)

    model = LogisticRegression(multi_class="ovr", max_iter=1000)
    model.fit(X_train, y_train)
    prediction = model.predict(X_test)
    print(np.round(100 * accuracy_score(y_test, prediction), 2))

c:\Users\dipak\.conda\envs\env2\Lib\site-packages\sklearn\linear_model\_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Users\dipak\.conda\envs\env2\Lib\site-packages\sklearn\linear_model\_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


81.13
75.08


In [40]:
for i in [0.1, 0.3]:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=i, stratify=y)

    model = OneVsOneClassifier(LogisticRegression(max_iter=1000))
    model.fit(X_train, y_train)
    prediction = model.predict(X_test)
    print(np.round(100 * accuracy_score(y_test, prediction), 2))

92.45
94.16


In [ ]:
def obesity_risk_pipeline(data_path, test_size=0.2):
    try:
        df = pd.read_csv(data_path)
    except Exception as e:
        return f"error as {e}"

    # Standardizing the continous numerical features:
    continous_columns = df.select_dtypes(include=["float64"]).columns.tolist()
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(df[continous_columns])
    scaled_features_df = pd.DataFrame(
        scaled_features, columns=scaler.get_feature_names_out(continous_columns)
    )
    # Combining with the original dataset
    scaled_data = pd.concat(
        [df.drop(columns=continous_columns), scaled_features_df], axis=1
    )
    categorical_columns = scaled_data.select_dtypes(
        include=["object"]
    ).columns.to_list()
    categorical_columns.remove("NObeyesdad")
    # Applying one hot encoding
    encoder = OneHotEncoder(sparse_output=False, drop="first")
    encoded_features = encoder.fit_transform(scaled_data[categorical_columns])
    encoded_df = pd.DataFrame(
        encoded_features, columns=encoder.get_feature_names_out(categorical_columns)
    )
    prepped_data = pd.concat(
        [scaled_data.drop(columns=categorical_columns), encoded_df], axis=1
    )
    prepped_data["NObeyesdad"] = prepped_data["NObeyesdad"].astype("category").cat.codes
    X = prepped_data.drop("NObeyesdad", axis=1)
    y = prepped_data["NObeyesdad"]
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=42, stratify=y
    )
    model_ova = LogisticRegression(multi_class="multinomial", max_iter=1000)
    model_ova.fit(X_train, y_train)
    y_pred_ova = model_ova.predict(X_test)
    print("One-vs-all (OvA) Strategy")
    print(f"Accuracy: {np.round(100*accuracy_score(y_test,y_pred_ova),2)}%")
    model_ovo = OneVsOneClassifier(LogisticRegression(max_iter=1000))
    model_ovo.fit(X_train, y_train)
    y_pred_ovo = model_ovo.predict(X_test)
    print("One-vs-One  (OvO) strategy")
    print(f"Accuracy: {np.round(100*accuracy_score(y_test, y_pred_ovo),2)}%")


obesity_risk_pipeline(file_path, test_size=0.2)